# Overview

- HashiCorp is a company who's "TAO" is workflow
    - terraform: code once for all code providers

- Phases
    - Define (config files)
    - Plan (preview the setup)
    - Apply (apply the setup)
    - Update (make changes and apply)

- Terraform creates reproducible infrastructure
    - Includes different environments
        - development
        - staging
        - production

- Modules encapsulate a set of configurations

- Hybrid infrastructure

- Managing resources requires "ordering" which terraform handles well automatically
    - via terraform's "interpolation"
    
- Besides cloud providers, terraform works with:
    -75 Integrations in terraform core
        - DIN
        - kubernetes
        - puppet
        - consul
        - github
            - see blog post
            
- Google Cloud
    - 

## Examples
### Composing Resources

Use attributes from other resources to create an infrastructure composed of resources across multiple providers.


`
resource "digitalocean_droplet" "web" {
  name   = "tf-web"
  size   = "512mb"
  image  = "centos-5-8-x32"
  region = "sfo1"
}
`

`
resource "dnsimple_record" "hello" {
  domain = "example.com"
  name   = "test"
  value  = "${digitalocean_droplet.web.ipv4_address}"
  type   = "A"
}
`

### Fast Simplified Interaction

Simple and intuitive configuration makes even the most complicated services approachable: no more web consoles, loading bars, or confusing CLI clients.


`
resource "aws_elb" "frontend" {
  name = "frontend-load-balancer"
  listener {
    instance_port     = 8000
    instance_protocol = "http"
    lb_port           = 80
    lb_protocol       = "http"
  }
  instances = ["${aws_instance.app.*.id}"]
}
` 

`
resource "aws_instance" "app" {
    count = 5
    ami           = "ami-408c7f28"
    instance_type = "t1.micro"
}
`

## Google Cloud Platform Walkthrough

`
provider "google" {
    region = "us-central1"
    credentials = "${var.billing_account}"
}
`

`
resource "google_project" "demo" {
    project_id = "tf-demo-${var.user}"
    name = "Demo Project"
    org_id = "${var.organization_id}"
    billing_account = "${var.billing_account}"
}
`

`
resource "google_project_services" "apis" {
    project = "${google_project.demo.project_id}"
    services = ["compute-component.googleapis.com"]
}
`

`resource "google_compute_instance" "vm" {
    project = "${google_project.demo.project_id}"
    name = "dev-instance"
    machine_type = "n1-standard-1"
    zone = "us-central1-f"
    disk {
        image = "debian-cloud/debian-8"
    }
    network_interface {
        network = "default"
        access_config {
            // Ephemeral IP
        }
     }
     depends_on = ["google_project_services.apis"]
}
`